<a href="https://colab.research.google.com/github/k1ran242/kiran/blob/main/Spam_Classifier_Using_LSTM_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
data=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Natural Language Kaggle/NLP_train.csv')
data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [3]:
data.dropna()

,id,keyword,location,text,target
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0
...,...,...,...,...,...
7575,10826,wrecked,TN,On the bright side I wrecked http://t.co/uEa0t...,0
7577,10829,wrecked,#NewcastleuponTyne #UK,@widda16 ... He's gone. You can relax. I thoug...,0
7579,10831,wrecked,"Vancouver, Canada",Three days off from work and they've pretty mu...,0
7580,10832,wrecked,London,#FX #forex #trading Cramer: Iger's 3 words tha...,0


In [4]:
data.reset_index(inplace=True)

In [5]:
df=data.drop(['id','keyword','location'],axis=1)

In [6]:
X=df['text']
y=df['target']

In [7]:
X.shape

(7613,)

In [8]:
y.shape

(7613,)

In [9]:
import tensorflow as tf
tf.__version__

'2.5.0'

In [10]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense


In [11]:
voc_size=5000

**OneHot Representation**

In [12]:
messages=X.copy()

In [13]:
import nltk
import re
from nltk.corpus import stopwords

In [14]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
## Data Preprocessing
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [16]:
corpus=[]
for i in range(0,len(messages)):
  text=re.sub('[^a-zA-Z]',' ',messages[i])
  text=text.lower()
  text=text.split()
  text=[ps.stem(word) for word in text if not word in stopwords.words('english')]
  text=' '.join(text)
  corpus.append(text)

In [17]:
#corpus

In [18]:
onehot_repr=[one_hot(words,voc_size) for words in corpus]
onehot_repr[0]

[4671, 4490, 1706, 1764, 1656, 425, 178]

**Embedding Representation**

In [19]:
sent_lenght=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_lenght)
embedded_docs[100]

array([   0,    0,    0,    0,    0,    0,    0,    0, 3125, 2653, 2637,
        858, 2880,  680,  469, 3510,   96, 3456,  745, 2680], dtype=int32)

In [20]:
len(embedded_docs)

7613

**Creating Models**

In [21]:
embedding_vector_feature=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_feature,input_length=sent_lenght))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [23]:
len(embedded_docs),y.shape

(7613, (7613,))

In [24]:
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [25]:
X_final.shape,y_final.shape

((7613, 20), (7613,))

In [26]:
# from sklearn.model_selection import train_test_split
# x_train,x_test,y_train,y_test = train_test_split(X_final,y_final,test_size=0.25,random_state=101)

**Model Training**

In [27]:
model.fit(X_final,y_final,epochs=20,batch_size=32)

Epoch 1/20
238/238 [==============================] - 4s 7ms/step - loss: 0.5397 - accuracy: 0.7232
Epoch 2/20
238/238 [==============================] - 2s 7ms/step - loss: 0.3688 - accuracy: 0.8461
Epoch 3/20
238/238 [==============================] - 2s 7ms/step - loss: 0.2936 - accuracy: 0.8835
Epoch 4/20
238/238 [==============================] - 2s 7ms/step - loss: 0.2404 - accuracy: 0.9065
Epoch 5/20
238/238 [==============================] - 2s 7ms/step - loss: 0.1830 - accuracy: 0.9318
Epoch 6/20
238/238 [==============================] - 2s 7ms/step - loss: 0.1347 - accuracy: 0.9543
Epoch 7/20
238/238 [==============================] - 2s 7ms/step - loss: 0.0994 - accuracy: 0.9677
Epoch 8/20
238/238 [==============================] - 2s 7ms/step - loss: 0.0852 - accuracy: 0.9724
Epoch 9/20
238/238 [==============================] - 2s 7ms/step - loss: 0.0653 - accuracy: 0.9787
Epoch 10/20
238/238 [==============================] - 2s 7ms/step - loss: 0.0589 - accuracy: 0.9804

In [28]:
test_data=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Natural Language Kaggle/NLP_test.csv')
test_data.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [29]:
test_data.dropna()

,id,keyword,location,text
15,46,ablaze,London,Birmingham Wholesale Market is ablaze BBC News...
16,47,ablaze,Niall's place | SAF 12 SQUAD |,@sunkxssedharry will you wear shorts for race ...
17,51,ablaze,NIGERIA,#PreviouslyOnDoyinTv: Toke MakinwaÛªs marriag...
18,58,ablaze,Live On Webcam,Check these out: http://t.co/rOI2NSmEJJ http:/...
19,60,ablaze,"Los Angeles, Califnordia",PSA: IÛªm splitting my personalities.\n\n?? t...
...,...,...,...,...
3246,10804,wrecked,Love Reiss,@yakubOObs think he deactivated because his no...
3247,10806,wrecked,Seattle Washington,RT CNBC '3 words from Disney CEO Bob Iger wrec...
3248,10807,wrecked,Acey mountain islanddåÇTorontoåÈ,Smackdown tyme this should put me in a good mo...
3249,10816,wrecked,los angeles,@thrillhho jsyk I haven't stopped thinking abt...


In [30]:
test_data.reset_index(inplace=True)

In [31]:
test_df=test_data.drop(['id','keyword','location'],axis=1)

In [32]:
test_X=test_df['text']

In [33]:
test_messages=test_X.copy()

In [34]:
test_corpus=[]
for i in range(0,len(test_messages)):
  text=re.sub('[^a-zA-Z]',' ',test_messages[i])
  text=text.lower()
  text=text.split()
  text=[ps.stem(word) for word in text if not word in stopwords.words('english')]
  text=' '.join(text)
  test_corpus.append(text)

In [35]:
onehot_repr_test=[one_hot(words,voc_size) for words in test_corpus]
onehot_repr_test[0]

[4430, 3245, 4827, 809]

In [36]:
sent_lenght=20
embedded_docs_test=pad_sequences(onehot_repr_test,padding='pre',maxlen=sent_lenght)
embedded_docs_test[100]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0, 1202,  849, 2592, 3456,  745, 1891, 2886,  224], dtype=int32)

In [37]:
len(embedded_docs_test)

3263

In [38]:
X_final_test=np.array(embedded_docs_test)

In [39]:
print(X_final_test.shape)

(3263, 20)


In [40]:
y_pred=model.predict_classes(X_final_test)
y_pred

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([[1],
       [0],
       [1],
       ...,
       [0],
       [1],
       [0]], dtype=int32)

In [41]:
pred=pd.DataFrame(y_pred)
subm=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Natural Language Kaggle/sample_submission_NLP.csv')

In [42]:
dataset=pd.concat([subm['id'],pred],axis=1)
dataset.columns=["id",'target']

In [43]:
dataset.to_csv("NLP_kaggale_competation4.csv",index=False)